In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pickle
from time import time
from sklearn.decomposition import PCA
import george
from george import kernels
from scipy.optimize import minimize
import argparse


In [3]:
rname="trans4"
wname="trans4_verbose"
verbose=True

In [4]:
# load cube
with open ("./gmd/cubeflat.txt","rb") as fp:
	cubeflat=pickle.load(fp)

# load pca weight data generated by get_pca_weights.py
with open ("./gmd/"+rname+"_parvals.txt","rb") as fp:
	Xs=pickle.load(fp)
with open ("./gmd/"+rname+"_weights.txt","rb") as fp:
	ws=pickle.load(fp)
with open ("./gmd/"+rname+"_eigenseds.txt","rb") as fp:
	eigenseds=pickle.load(fp)
with open ("./gmd/"+rname+"_mean.txt","rb") as fp:
	pcamean=pickle.load(fp)
print("pca weights loaded from "+rname)

pca weights loaded from trans4


In [5]:
yerrs=[]
for i in range(16):
	yerrs.append([x*0.01 for x in ws[i]])
initvecs=[]
for i in range(16):
    initvecs.append([ 6.33043185, 18.42068074,  0.,  0., -0.4462871 , -5.05145729, -1.38629436, 
                     -2.4079456086518722, -2.4079456086518722,  -3.2188758248682006, -3.21887582, 
                     -2.77258872, -2.77258872,  1.38629436,  2.19722458, 0.8109302162163288])


In [8]:
np.array(initvecs).shape

(16, 16)

In [9]:
kernel = 23*kernels.ExpSquaredKernel(1**2,ndim=15,axes=0)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=1)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=2)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=3)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=4)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=5)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=6)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=7)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=8)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=9)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=10)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=11)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=12)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=13)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=14) 
blankhodlr=george.GP(kernel,solver=george.HODLRSolver)


In [18]:
def F_chisq_quiet(hp,gp):
    hyperparams=np.array(hp).reshape(16,16)
    preds=[]
    for i in range(len(ws)):  # same covfunc for each weight and the sample mean
        t1=time()
        gp.set_parameter_vector(hyperparams[i])
        if verbose==True:
            print("weight #"+str(i))
            print(gp.get_parameter_vector())
        gp.compute(Xs,yerrs[i])
        t2=time()
        pred, pred_var = gp.predict(ws[i], Xs, return_var=True)
        preds.append(pred)
    reconst_SEDs=[]
    for i in range(3850):
        reconst=np.dot(np.array(preds)[:,i][0:15],eigenseds[0:15]) + pcamean + np.array(preds)[:,i][15]
        reconst_SEDs.append(reconst)
    allsedsflat=np.ndarray.flatten(np.array(reconst_SEDs))
    chisq=np.sum((cubeflat-allsedsflat)**2/0.1)
    if verbose==True:
        print(chisq)
    return chisq

def chisq(p):
    return F_chisq_quiet(p,blankhodlr)


In [19]:
minimize(chisq,initvecs)

(16, 16)
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.3304318

KeyboardInterrupt: 

In [20]:
F_chisq_quiet(initvecs,blankhodlr)

(16, 16)
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.33043185 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436 -2.40794561 -2.40794561 -3.21887582 -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  0.81093022]
[ 6.3304318

5767562.735884244